In [32]:
# importing pre-requisite libraries
import os
import sys
import pandas as pd
import numpy as np

# Get the absolute path of the credit_risk package
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)
print(project_root)


c:\Aravind's File\Projects\credit_risk


In [33]:
#importing the modules from the package
from pipeline.common_pipeline import CommonFunctions

In [34]:
common_functions = CommonFunctions()
config_ = common_functions.config_
data_path_list = common_functions.data_path_list

In [35]:
feature_data = common_functions.read_data(data_path_list['path'], data_path_list['feature_engineered_data'], 'parquet')

# Model exploration with imbalanced Dataset

In [36]:
feature_data

,Age,Credit amount,Duration,Risk,Job_1,Job_2,Job_3,Housing_own,Housing_rent,Saving accounts_little,...,Checking account_moderate,Checking account_rich,Purpose_car,Purpose_domestic appliances,Purpose_education,Purpose_furniture/equipment,Purpose_radio/TV,Purpose_repairs,Purpose_vacation/others,Sex_male
0,2.766456,-0.745131,-1.236478,good,False,True,False,True,False,False,...,False,False,False,False,False,False,True,False,False,True
1,-1.191404,0.949817,2.248194,bad,False,True,False,True,False,True,...,True,False,False,False,False,False,True,False,False,False
2,1.183312,-0.416562,-0.738668,good,True,False,False,True,False,True,...,False,False,False,False,True,False,False,False,False,True
3,0.831502,1.634247,1.750384,good,False,True,False,False,False,True,...,False,False,False,False,False,True,False,False,False,True
4,1.535122,0.566664,0.256953,bad,False,True,False,False,False,True,...,False,False,True,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-0.399832,-0.544162,-0.738668,good,True,False,False,True,False,True,...,False,False,False,False,False,True,False,False,False,False
996,0.391740,0.207612,0.754763,good,False,False,True,True,False,True,...,False,False,True,False,False,False,False,False,False,True
997,0.215835,-0.874503,-0.738668,good,False,True,False,True,False,True,...,False,False,False,False,False,False,True,False,False,True
998,-1.103451,-0.505528,1.999289,bad,False,True,False,False,False,True,...,False,False,False,False,False,False,True,False,False,True


In [37]:
# Calculate percentages
target_percentage = feature_data['Risk'].value_counts(normalize=True) * 100
target_percentage

Risk
good    70.0
bad     30.0
Name: proportion, dtype: float64

We have imbalanced dataset in our data, however we build a model with this imbalanced data and check how the models will perform

In [38]:
#Label Encoding
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

X = feature_data.drop('Risk', axis=1)
y = feature_data['Risk']

# Encode the target variable
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Check the mapping
print("Label Mapping:", dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))



Label Mapping: {'bad': np.int64(0), 'good': np.int64(1)}


In [39]:
# Split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Print the shapes of the resulting datasets
print("Training Features Shape:", X_train.shape)
print("Testing Features Shape:", X_test.shape)
print("Training Target Shape:", y_train.shape)
print("Testing Target Shape:", y_test.shape)

Training Features Shape: (800, 23)
Testing Features Shape: (200, 23)
Training Target Shape: (800,)
Testing Target Shape: (200,)


In [40]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier, RidgeClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

In [41]:
# Define a dictionary of models to test
models = {
    "Logistic Regression": LogisticRegression(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "AdaBoost": AdaBoostClassifier(random_state=42),
    "XGBoost": XGBClassifier(random_state=42),
    "Support Vector Machine": SVC(random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Naive Bayes": GaussianNB(),
    "SGD Classifier": SGDClassifier(random_state=42),
    "Ridge Classifier": RidgeClassifier(random_state=42)
}


In [42]:
results = []
for model_name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)

    # Make predictions
    y_test_pred = model.predict(X_test)

    y_train_pred = model.predict(X_train)

    # Calculate metrics
    accuracy_train = accuracy_score(y_train, y_train_pred)
    accuracy_test = accuracy_score(y_test, y_test_pred)
    precision = precision_score(y_test, y_test_pred)
    recall = recall_score(y_test, y_test_pred)
    f1 = f1_score(y_test, y_test_pred)

    # Save the results
    results.append({
        "Model": model_name,
        "Accuracy_train": accuracy_train,
        "Accuracy_test": accuracy_test,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1
    })

In [43]:
results_df = pd.DataFrame(results)
results_df

,Model,Accuracy_train,Accuracy_test,Precision,Recall,F1 Score
0,Logistic Regression,0.74500,0.740,0.785714,0.864286,0.823129
1,Random Forest,1.00000,0.745,0.794702,0.857143,0.824742
2,Gradient Boosting,0.88250,0.760,0.783951,0.907143,0.841060
3,AdaBoost,0.75500,0.755,0.782609,0.900000,0.837209
4,XGBoost,1.00000,0.780,0.807692,0.900000,0.851351
5,Support Vector Machine,0.79625,0.770,0.773256,0.950000,0.852564
6,Decision Tree,1.00000,0.700,0.798507,0.764286,0.781022
7,K-Nearest Neighbors,0.78750,0.695,0.736527,0.878571,0.801303
8,Naive Bayes,0.68625,0.680,0.783582,0.750000,0.766423
9,SGD Classifier,0.73500,0.725,0.745665,0.921429,0.824281


## Testing the following support vector machine, gradient boosting and xgboosting for hyper parameter tuning and finalise the model

### Model 1 Support Vector Machine

In [44]:

# hyper parameter tuning for SVM
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 10, 100],  # Regularization parameter
    'kernel': ['linear', 'rbf', 'poly'],  # Kernel types
    'gamma': [1, 0.1, 0.01, 0.001],  # Kernel coefficient
    'degree': [2, 3, 4]  # Only relevant for 'poly' kernel
}

# Initialize the SVM model
svm = SVC(random_state=42)

# Perform GridSearchCV
grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Accuracy:", grid_search.best_score_)

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
y_test_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

Fitting 5 folds for each of 144 candidates, totalling 720 fits
Best Parameters: {'C': 10, 'degree': 2, 'gamma': 1, 'kernel': 'linear'}
Best Cross-Validation Accuracy: 0.7324999999999999
Test Accuracy: 0.725


In [45]:
# Access all results from GridSearchCV
grid_search_results = grid_search.cv_results_

# Convert results to a DataFrame for better readability
import pandas as pd
grid_search_results_df = pd.DataFrame(grid_search_results)

# Display the top rows of the DataFrame
print("All GridSearchCV Results:")
print(grid_search_results_df.head())

# # Optional: Display specific columns for better clarity
# columns_to_display = [
#     'param_C', 'param_kernel', 'param_gamma', 'param_degree', 
#     'mean_test_score', 'std_test_score', 'rank_test_score'
# ]
print("\nSelected Results:")
print(grid_search_results_df.sort_values(by='rank_test_score'))
grid_search_results_df.to_csv('svm_grid_search_results.csv', index=False)

All GridSearchCV Results:
   mean_fit_time  std_fit_time  mean_score_time  std_score_time  param_C  \
0       0.015845      0.001401         0.008421        0.007570      0.1   
1       0.028183      0.006203         0.015138        0.002355      0.1   
2       0.021754      0.002366         0.005513        0.001584      0.1   
3       0.014842      0.001370         0.004510        0.000630      0.1   
4       0.020357      0.002812         0.010423        0.001533      0.1   

   param_degree  param_gamma param_kernel  \
0             2          1.0       linear   
1             2          1.0          rbf   
2             2          1.0         poly   
3             2          0.1       linear   
4             2          0.1          rbf   

                                              params  split0_test_score  \
0  {'C': 0.1, 'degree': 2, 'gamma': 1, 'kernel': ...            0.74375   
1  {'C': 0.1, 'degree': 2, 'gamma': 1, 'kernel': ...            0.70000   
2  {'C': 0.1, 'degree

### Model 2 Gradient Boosting

In [46]:
# hyper parameter tuning for Gradient Boosting
# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],  # Number of trees
    'learning_rate': [0.01, 0.1, 0.2],  # Learning rate
    'max_depth': [3, 5, 7],  # Maximum depth of trees
    'min_samples_split': [2, 5, 10],  # Minimum samples to split a node
    'min_samples_leaf': [1, 2, 4]  # Minimum samples in a leaf node
}



In [47]:
# Initialize the Gradient Boosting model
gb = GradientBoostingClassifier(random_state=42)

# Perform GridSearchCV
grid_search_gb = GridSearchCV(estimator=gb, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search_gb.fit(X_train, y_train)


Fitting 5 folds for each of 243 candidates, totalling 1215 fits


GridSearchCV(cv=5, estimator=GradientBoostingClassifier(random_state=42),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7], 'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]},
             scoring='accuracy', verbose=1)

In [48]:
# Print the best parameters and best score
print("Best Parameters:", grid_search_gb.best_params_)
print("Best Cross-Validation Accuracy:", grid_search_gb.best_score_)

# Evaluate the best model on the test set
best_gb_model = grid_search_gb.best_estimator_
y_test_pred_gb = best_gb_model.predict(X_test)

# Calculate test accuracy
from sklearn.metrics import accuracy_score
test_accuracy_gb = accuracy_score(y_test, y_test_pred_gb)
print("Test Accuracy:", test_accuracy_gb)

Best Parameters: {'learning_rate': 0.1, 'max_depth': 3, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100}
Best Cross-Validation Accuracy: 0.75
Test Accuracy: 0.745


In [50]:
# Access all results from GridSearchCV
grid_search_results = grid_search_gb.cv_results_

# Convert results to a DataFrame for better readability
import pandas as pd
grid_search_results_df = pd.DataFrame(grid_search_results)

# Display the top rows of the DataFrame
print("All GridSearchCV Results:")
print(grid_search_results_df.head())

# # Optional: Display specific columns for better clarity
# columns_to_display = [
#     'param_C', 'param_kernel', 'param_gamma', 'param_degree', 
#     'mean_test_score', 'std_test_score', 'rank_test_score'
# ]
print("\nSelected Results:")
print(grid_search_results_df.sort_values(by='rank_test_score'))
grid_search_results_df.to_csv('gradient_boosting_grid_search_results.csv', index=False)

All GridSearchCV Results:
   mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0       0.103902      0.010225         0.006620        0.007778   
1       0.190056      0.003658         0.003303        0.000875   
2       0.367537      0.013600         0.003005        0.000006   
3       0.097684      0.004020         0.002511        0.000319   
4       0.190349      0.007049         0.002503        0.000448   

   param_learning_rate  param_max_depth  param_min_samples_leaf  \
0                 0.01                3                       1   
1                 0.01                3                       1   
2                 0.01                3                       1   
3                 0.01                3                       1   
4                 0.01                3                       1   

   param_min_samples_split  param_n_estimators  \
0                        2                  50   
1                        2                 100   
2                 

### Model 3 Xgboost

In [51]:
# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],  # Number of trees
    'learning_rate': [0.01, 0.1, 0.2],  # Learning rate
    'max_depth': [3, 5, 7],  # Maximum depth of trees
    'min_child_weight': [1, 3, 5],  # Minimum sum of weights in a child node
    'subsample': [0.6, 0.8, 1.0],  # Fraction of samples used per tree
    'colsample_bytree': [0.6, 0.8, 1.0]  # Fraction of features used per tree
}

# Initialize the XGBoost model
xgb = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')

# Perform GridSearchCV
grid_search_xgb = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search_xgb.fit(X_train, y_train)



Fitting 5 folds for each of 729 candidates, totalling 3645 fits


c:\Users\aravi\anaconda3\envs\credit_risk_env\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:13:16] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='logloss', feature_types=None,
                                     feature_weights=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraint...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.6, 0.8, 1.0],
                         'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7], 'min_child_weight': [1, 3, 5],
                         'n_estimators': [50, 100, 200],
                         'subsample': [0.6, 0.8, 1.0]},
             scoring='accuracy', verbose=1)

In [52]:
# Print the best parameters and best score
print("Best Parameters:", grid_search_xgb.best_params_)
print("Best Cross-Validation Accuracy:", grid_search_xgb.best_score_)

# Evaluate the best model on the test set
best_xgb_model = grid_search_xgb.best_estimator_
y_test_pred_xgb = best_xgb_model.predict(X_test)

# Calculate test accuracy
from sklearn.metrics import accuracy_score
test_accuracy_xgb = accuracy_score(y_test, y_test_pred_xgb)
print("Test Accuracy:", test_accuracy_xgb)

Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 200, 'subsample': 1.0}
Best Cross-Validation Accuracy: 0.74875
Test Accuracy: 0.765


As of now the Cross validation accuracy of all models stands at
        Cross Validation Accuracy| Test Acuracy
1. SVM -                73.2%    | 72.5%
2. Gradient Boosting -  75%      | 74.5%
3. Xgboost -            74.8%    | 76.5%

# Model exploration with Balanced Dataset

In [53]:
X.shape

(1000, 23)

In [54]:
y.shape

(1000,)

In [55]:
from imblearn.over_sampling import SMOTE

# Step 1: Oversample the minority class using SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 2: Split the balanced dataset into training and testing sets
X_train_balanced, X_test_balanced, y_train_balanced, y_test_balanced = train_test_split(
    X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled
)

In [56]:
X_resampled.shape

(1400, 23)

In [57]:
# Count unique values and their occurrences
unique_values, counts = np.unique(y_resampled, return_counts=True)
counts

array([700, 700])

In [58]:
results_balanced = []
for model_name, model in models.items():
    # Train the model
    model.fit(X_train_balanced, y_train_balanced)

    # Make predictions
    y_test_pred = model.predict(X_test_balanced)

    y_train_pred = model.predict(X_train_balanced)

    # Calculate metrics
    accuracy_train = accuracy_score(y_train_balanced, y_train_pred)
    accuracy_test = accuracy_score(y_test_balanced, y_test_pred)
    precision = precision_score(y_test_balanced, y_test_pred)
    recall = recall_score(y_test_balanced, y_test_pred)
    f1 = f1_score(y_test_balanced, y_test_pred)

    # Save the results
    results_balanced.append({
        "Model": model_name,
        "Accuracy_train": accuracy_train,
        "Accuracy_test": accuracy_test,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1
    })

In [59]:
results_balanced_df = pd.DataFrame(results_balanced)
results_balanced_df

,Model,Accuracy_train,Accuracy_test,Precision,Recall,F1 Score
0,Logistic Regression,0.754464,0.746429,0.759398,0.721429,0.739927
1,Random Forest,1.000000,0.800000,0.833333,0.750000,0.789474
2,Gradient Boosting,0.861607,0.767857,0.790698,0.728571,0.758364
3,AdaBoost,0.769643,0.764286,0.789062,0.721429,0.753731
4,XGBoost,1.000000,0.835714,0.861538,0.800000,0.829630
5,Support Vector Machine,0.842857,0.792857,0.825397,0.742857,0.781955
6,Decision Tree,1.000000,0.735714,0.753846,0.700000,0.725926
7,K-Nearest Neighbors,0.853571,0.775000,0.818182,0.707143,0.758621
8,Naive Bayes,0.691964,0.710714,0.697987,0.742857,0.719723
9,SGD Classifier,0.722321,0.700000,0.811111,0.521429,0.634783


In [61]:
results_df

,Model,Accuracy_train,Accuracy_test,Precision,Recall,F1 Score
0,Logistic Regression,0.74500,0.740,0.785714,0.864286,0.823129
1,Random Forest,1.00000,0.745,0.794702,0.857143,0.824742
2,Gradient Boosting,0.88250,0.760,0.783951,0.907143,0.841060
3,AdaBoost,0.75500,0.755,0.782609,0.900000,0.837209
4,XGBoost,1.00000,0.780,0.807692,0.900000,0.851351
5,Support Vector Machine,0.79625,0.770,0.773256,0.950000,0.852564
6,Decision Tree,1.00000,0.700,0.798507,0.764286,0.781022
7,K-Nearest Neighbors,0.78750,0.695,0.736527,0.878571,0.801303
8,Naive Bayes,0.68625,0.680,0.783582,0.750000,0.766423
9,SGD Classifier,0.73500,0.725,0.745665,0.921429,0.824281


1. From the results above between imbalanced and balanced datasets we can understand that model test accuracy and precision have been slightly reduced but we see good improvement in recall and this results in good f1 score.
2. Precision focuses on the quality of positive predictions and recall focuses on the quantity of positive predictions

## Testing the following support vector machine, gradient boosting and xgboosting for hyper parameter tuning and finalise the model

### Model 1 exploration with svm for balanced data

In [62]:
# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 10, 100],  # Regularization parameter
    'kernel': ['linear', 'rbf', 'poly'],  # Kernel types
    'gamma': [1, 0.1, 0.01, 0.001],  # Kernel coefficient
    'degree': [2, 3, 4]  # Only relevant for 'poly' kernel
}

# Initialize the SVM model
svm = SVC(random_state=42)

# Perform GridSearchCV
grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train_balanced, y_train_balanced)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Accuracy:", grid_search.best_score_)

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
y_test_pred = best_model.predict(X_test_balanced)
test_accuracy = accuracy_score(y_test_balanced, y_test_pred)
print("Test Accuracy:", test_accuracy)

Fitting 5 folds for each of 144 candidates, totalling 720 fits
Best Parameters: {'C': 1, 'degree': 2, 'gamma': 1, 'kernel': 'rbf'}
Best Cross-Validation Accuracy: 0.8044642857142857
Test Accuracy: 0.8357142857142857


In [63]:
# Access all results from GridSearchCV
grid_search_results = grid_search.cv_results_

# Convert results to a DataFrame for better readability
import pandas as pd
grid_search_results_df = pd.DataFrame(grid_search_results)

# Display the top rows of the DataFrame
print("All GridSearchCV Results:")
print(grid_search_results_df.head())

# # Optional: Display specific columns for better clarity
# columns_to_display = [
#     'param_C', 'param_kernel', 'param_gamma', 'param_degree', 
#     'mean_test_score', 'std_test_score', 'rank_test_score'
# ]
print("\nSelected Results:")
print(grid_search_results_df.sort_values(by='rank_test_score'))
grid_search_results_df.to_csv('svm_balanced_grid_search_results.csv', index=False)

All GridSearchCV Results:
   mean_fit_time  std_fit_time  mean_score_time  std_score_time  param_C  \
0       0.032868      0.008393         0.007121        0.000662      0.1   
1       0.042690      0.004613         0.029278        0.005102      0.1   
2       0.037681      0.002770         0.008623        0.001719      0.1   
3       0.027759      0.002303         0.007619        0.000864      0.1   
4       0.044100      0.006216         0.028196        0.005627      0.1   

   param_degree  param_gamma param_kernel  \
0             2          1.0       linear   
1             2          1.0          rbf   
2             2          1.0         poly   
3             2          0.1       linear   
4             2          0.1          rbf   

                                              params  split0_test_score  \
0  {'C': 0.1, 'degree': 2, 'gamma': 1, 'kernel': ...           0.669643   
1  {'C': 0.1, 'degree': 2, 'gamma': 1, 'kernel': ...           0.714286   
2  {'C': 0.1, 'degree

### Model 2 Gradient Boosting

In [64]:
# hyper parameter tuning for Gradient Boosting
# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],  # Number of trees
    'learning_rate': [0.01, 0.1, 0.2],  # Learning rate
    'max_depth': [3, 5, 7],  # Maximum depth of trees
    'min_samples_split': [2, 5, 10],  # Minimum samples to split a node
    'min_samples_leaf': [1, 2, 4]  # Minimum samples in a leaf node
}



In [65]:
# Initialize the Gradient Boosting model
gb = GradientBoostingClassifier(random_state=42)

# Perform GridSearchCV
grid_search_gb = GridSearchCV(estimator=gb, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search_gb.fit(X_train_balanced, y_train_balanced)


Fitting 5 folds for each of 243 candidates, totalling 1215 fits


GridSearchCV(cv=5, estimator=GradientBoostingClassifier(random_state=42),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7], 'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]},
             scoring='accuracy', verbose=1)

In [66]:
# Print the best parameters and best score
print("Best Parameters:", grid_search_gb.best_params_)
print("Best Cross-Validation Accuracy:", grid_search_gb.best_score_)

# Evaluate the best model on the test set
best_gb_model = grid_search_gb.best_estimator_
y_test_pred_gb = best_gb_model.predict(X_test_balanced)

# Calculate test accuracy
from sklearn.metrics import accuracy_score
test_accuracy_gb = accuracy_score(y_test_balanced, y_test_pred_gb)
print("Test Accuracy:", test_accuracy_gb)

Best Parameters: {'learning_rate': 0.2, 'max_depth': 7, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 50}
Best Cross-Validation Accuracy: 0.7875000000000001
Test Accuracy: 0.7964285714285714


### Model 3 Xgboost

In [67]:
# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],  # Number of trees
    'learning_rate': [0.01, 0.1, 0.2],  # Learning rate
    'max_depth': [3, 5, 7],  # Maximum depth of trees
    'min_child_weight': [1, 3, 5],  # Minimum sum of weights in a child node
    'subsample': [0.6, 0.8, 1.0],  # Fraction of samples used per tree
    'colsample_bytree': [0.6, 0.8, 1.0]  # Fraction of features used per tree
}

# Initialize the XGBoost model
xgb = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')

# Perform GridSearchCV
grid_search_xgb = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search_xgb.fit(X_train_balanced, y_train_balanced)



Fitting 5 folds for each of 729 candidates, totalling 3645 fits


c:\Users\aravi\anaconda3\envs\credit_risk_env\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:36:35] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='logloss', feature_types=None,
                                     feature_weights=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraint...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.6, 0.8, 1.0],
                         'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7], 'min_child_weight': [1, 3, 5],
                         'n_estimators': [50, 100, 200],
                         'subsample': [0.6, 0.8, 1.0]},
             scoring='accuracy', verbose=1)

In [68]:
# Print the best parameters and best score
print("Best Parameters:", grid_search_xgb.best_params_)
print("Best Cross-Validation Accuracy:", grid_search_xgb.best_score_)

# Evaluate the best model on the test set
best_xgb_model = grid_search_xgb.best_estimator_
y_test_pred_xgb = best_xgb_model.predict(X_test_balanced)

# Calculate test accuracy
from sklearn.metrics import accuracy_score
test_accuracy_xgb = accuracy_score(y_test_balanced, y_test_pred_xgb)
print("Test Accuracy:", test_accuracy_xgb)

Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 0.6}
Best Cross-Validation Accuracy: 0.7866071428571428
Test Accuracy: 0.8071428571428572


As of now the Cross validation accuracy of all models stands at
        Cross Validation Accuracy| Test Acuracy
1. SVM -                80.4%    | 83.5%
2. Gradient Boosting -  78.7%      | 79.6%
3. Xgboost -            78.6%    | 80.7%

# Final Summary

1. From the above analysis we can understand that balancing the datasets is very important for the model to perform well. 
2. The models are performing well on the balanced dataset. 
3. Among the models which we've explored SVM is providing better accuracy so we will be using it to build the final model